In [47]:
from skimage.io import collection, imread
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import re
import glob

from datetime import datetime

# How is the public opinion about a company correlated to it's market value?

A company's market value is variable and depends on a lot of factors. The price is a reflection of the company's perceived value – what the public is willing to pay for a piece of the company. It can and will rise and fall, based on a variety of factors in the global landscape and within the company itself. One of which is becoming more influential than ever - the public's opinion on social media.

To analyze this correlation we'll look at two datasets. The first one contains over 3 million unique tweets with their information such as tweet id, author of the tweet, post date, the text body of the tweet, and the number of comments, likes, and retweets of tweets matched with the related company.

The second one will just have daily stock price records (from the Forbes2000) for us to make a reference with.

### 1. Data Acquisition 

So first let's read the tweets dataset into pandas and inspect a small sample from the two dataframes.

In [48]:
tweets = pd.read_csv('./top-companies-tweets/Tweet.csv')
tweets.sample(5)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
2221194,955523054607765504,ElaineBenes99,1516649423,Bought $AAPL Feb. 2 $172.50 puts at 2.20 this ...,1,0,3
1590591,819096357923221506,sarnarahul,1484122764,Check Deadlines Directly from your #AppleWatch...,0,0,1
878545,709377473255907330,_peripherals,1457963743,"#Apple ( $AAPL), Xiaomi seeking India's DIPP n...",0,0,0
912217,714382346611339264,Sentifi_CA,1459156998,How #Tesla Motors could be profitable if it wa...,0,0,0
114991,571037018282569728,SkevaBoy,1424980809,"""@Uncanny_optomist: $AAPL oh wow is this remin...",0,0,0


In [49]:
tweets_company = pd.read_csv('./top-companies-tweets/Company_Tweet.csv')
tweets_company.sample(5)

,tweet_id,ticker_symbol
2863582,1212029798844977153,AMZN
1003240,957835765878349824,AAPL
2230311,653535637300076544,AMZN
1281176,1114191428857544707,AAPL
743095,804789781708185601,AAPL


So next up let's read the datasets for each of the stocks, which we are monitoring. We will save them in a dictionary with the key, being the company's tick name and the value - it's stock prices over time dataset.

In [50]:
stocks_df = {}
for name in glob.glob('./stocks/*'):
    stocks_df[name.split('\\')[-1].split('.')[0]] = pd.read_csv(name)
stocks = pd.concat(stocks_df)
stocks.sample(10)

Date          Low         Open     Volume         High  \
MSFT 3465  26-11-1999    45.500000    45.812500   28514200    46.687500   
AAPL 1896  14-06-1988     0.401786     0.404018  292420800     0.410714   
TSLA 1699  29-03-2017    55.108002    55.667999   18381000    55.919998   
AAPL 8234  06-08-2013    16.506071    16.715000  334857600    16.853214   
GOOG 3927  26-03-2020  1093.530029  1111.800049    3571700  1169.969971   
     2729  23-06-2015   535.250000   539.640015    1197500   541.499023   
TSLA 434   19-03-2012     6.908000     7.052000    5078000     7.064000   
MSFT 4536  04-03-2004    26.209999    26.330000   44023200    26.420000   
AAPL 3233  27-09-1993     0.216518     0.223214  113176000     0.225446   
     9048  27-10-2016    28.525000    28.847500  138248000    28.965000   

                 Close  Adjusted Close  
MSFT 3465    45.562500       28.872387  
AAPL 1896     0.404018        0.319994  
TSLA 1699    55.476002       55.476002  
AAPL 8234    16.616072       14.595078  
GOOG 3927  1161.750000     1161.750000  
     2729   540.479980      540.479980  
TSLA 434      6.996000        6.996000  
MSFT 4536    26.370001       16.858114  
AAPL 3233     0.220982        0.184605  
     9048    28.620001       26.881210

### 2. Data Tidying and Cleaning

First let's combine the two tables from the twitter dataset, convert the dates to a datetime object and rename the column.

In [51]:
tweets = pd.merge(tweets, tweets_company, on = "tweet_id")

In [54]:
tweets["date"] = pd.to_datetime(tweets.post_date, unit='s')
tweets = tweets.drop(columns="post_date")
tweets.sample(5)


,tweet_id,writer,body,comment_num,retweet_num,like_num,ticker_symbol,date
1012923,709425174240690176,aaplstocknews,Navigate the stock market with the #geneticalg...,0,0,0,TSLA,2016-03-14 17:05:16
48392,559773647980597248,racernic,$aapl playing with fire here,0,0,0,AAPL,2015-01-26 18:03:32
3435820,1080602683231354886,squawksquare,"During holiday Qtr, $AAPL makes about $1B/day....",3,3,9,AAPL,2019-01-02 23:12:10
3187545,1046462717916196865,PlainSite,"Musk agreed to ""comply with all mandatory proc...",0,0,2,TSLA,2018-09-30 18:12:08
3098636,1035560816056119296,ContrarianShort,$TSLA $TSLAQ Haven’t yet seen the FSD features...,4,0,10,TSLA,2018-08-31 16:11:52


We will see what timeframe does our dataset cover, by getting the data of the earliest and latest tweets.

In [ ]:
tweets.post_date.min(), tweets.post_date.max()

So it has data from 01.01.2015 to 31.12.2019, so basically from 2015 to the beginning of 2020. Knowing this we can filter out the stock prices to be only in this period of time. But first we have to covert the "Date" column to datetime.

In [55]:
def string_to_date(date_string):
    return datetime.strptime(date_string, "%d-%m-%Y")
stocks.Date = pd.to_datetime(stocks.Date.apply(string_to_date))

In [58]:
stocks = stocks[(stocks.Date >= '01-01-2015') & (stocks.Date < '01-01-2020')]
stocks.sample(10)

Date         Low        Open     Volume        High  \
AAPL 9177 2017-05-04   36.452499   36.630001   93487600   36.785000   
     8617 2015-02-12   31.392500   31.514999  297898000   31.870001   
     9065 2016-11-21   27.502501   27.530001  117058400   27.997499   
GOOG 2665 2015-03-23  554.308167  558.895569    1643801  560.820251   
MSFT 7774 2017-01-12   61.950001   63.060001   20968200   63.400002   
AAPL 9086 2016-12-21   29.195000   29.200001   95132800   29.350000   
MSFT 8363 2019-05-17  127.919998  128.309998   25770500  130.460007   
GOOG 3131 2017-01-26  827.010010  837.809998    2973900  838.000000   
AMZN 5113 2017-09-08  963.469971  979.099976    2605300  979.880005   
     4547 2015-06-11  431.470001  432.290009    2922500  438.890015   

                Close  Adjusted Close  
AAPL 9177   36.632500       34.733509  
     8617   31.615000       28.821255  
     9065   27.932501       26.370180  
GOOG 2665  557.279968      557.279968  
MSFT 7774   62.610001       58.431557  
AAPL 9086   29.264999       27.628145  
MSFT 8363  128.070007      125.299995  
GOOG 3131  832.150024      832.150024  
AMZN 5113  965.900024      965.900024  
     4547  432.970001      432.970001

As we can see we don't have data for every day, because the stock market funcitons only on workdays, unlike twitter.  We will find a way to work around this later.

Now lets strip the data down to just one column - the value, which we will calculate by getting the mean of the Open and Close prices. 

In [59]:
stocks["Value"] = (stocks.Open + stocks.Close) / 2
stocks = stocks.drop(columns=['Low', 'Open', 'Volume', 'High', 'Close', 'Adjusted Close'])
stocks.sample(10)

Date        Value
AAPL 8643 2015-03-23    31.791250
AMZN 5202 2018-01-17  1303.619995
TSLA 1782 2017-07-27    68.045998
GOOG 3324 2017-10-31  1015.929993
TSLA 1152 2015-01-27    41.039999
     2322 2019-09-19    49.260000
AAPL 9746 2019-08-08    50.453749
MSFT 8496 2019-11-25   150.614998
TSLA 1510 2016-06-28    40.368000
AAPL 9479 2018-07-17    47.650000

It is a little inconvenient to have the stock name as an index instead of it being a regular column. We will fix that and also change the column names to match the twitter dataset.

In [60]:
stocks = stocks.reset_index(level=0)

In [61]:
stocks.columns = ["ticker_symbol", "date", "value"]
stocks.sample(10)
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6290 entries, 8589 to 2393
Data columns (total 3 columns):
ticker_symbol    6290 non-null object
date             6290 non-null datetime64[ns]
value            6290 non-null float64
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 196.6+ KB


 So next up lets take a look at the datatypes and null values for the twitter dataset.

In [62]:
tweets.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4336445 entries, 0 to 4336444
Data columns (total 8 columns):
tweet_id         4336445 non-null int64
writer           4280526 non-null object
body             4336445 non-null object
comment_num      4336445 non-null int64
retweet_num      4336445 non-null int64
like_num         4336445 non-null int64
ticker_symbol    4336445 non-null object
date             4336445 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 297.8+ MB


Everything looks good, except the ticker_symbol which should be a category. The code below fixes that.

In [65]:
tweets.ticker_symbol = tweets.ticker_symbol.astype('category')